In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions['topics'] = questions.topics.apply(lambda s: s.split(','))
questions.iloc[0]

CPU times: user 20.7 s, sys: 4.7 s, total: 25.4 s
Wall time: 27.3 s


In [3]:
%%time
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer()
y_train = mb.fit_transform(questions.topics)
y_train.shape

CPU times: user 9.19 s, sys: 10.4 s, total: 19.6 s
Wall time: 23.2 s


In [4]:
colnames = ['id', 'parent_id', 'c', 'w', 'd_c', 'd_w']
topic_info = pd.read_csv('./topic_info.txt', sep='\t', names=colnames)

In [5]:
topic_words = [w for ws in topic_info.w for w in ws.split(',')]
print(len(topic_words))
print(len(set(topic_words)))

2693
2104


In [6]:
%%time
set_topic_words = set(topic_words)
questions['title_w'] = questions.title.astype('U').apply(lambda x: [i for i in x.split(',') if i in set_topic_words])

CPU times: user 10.9 s, sys: 1.41 s, total: 12.3 s
Wall time: 12.8 s


In [7]:
# questions.title_w[:5]
# todo: word distribution

0                               [w6, w109, w25, w1110]
1                [w377, w54, w57, w349, w54, w6, w875]
2                    [w875, w15450, w6, w25, w346, w6]
3                                 [w54, w138, w54, w6]
4    [w380, w54, w674, w133, w54, w134, w614, w54, ...
Name: title_w, dtype: object

In [7]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(questions.title_w.astype('U'))
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
print(X_train_tfidf.shape)

(2999967, 2090)
CPU times: user 23.2 s, sys: 655 ms, total: 23.9 s
Wall time: 24 s


In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

n_estimators = 10

In [11]:
%%time
svc = OneVsRestClassifier(LinearSVC())
svc.fit(X_train_tfidf[:1000], y_train[:1000])

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 11 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/p

CPU times: user 2.8 s, sys: 216 ms, total: 3.01 s
Wall time: 3.07 s


/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1892 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1893 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1894 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1898 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1901 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1904 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virt

In [12]:
from sklearn.metrics import accuracy_score
y = svc.predict(X_train_tfidf[:1000])
accuracy_score(y, y_train[:1000])

0.40000000000000002

In [ ]:
%%time
svc = OneVsRestClassifier(LinearSVC())
svc.fit(X_train_tfidf, y_train)

In [9]:
del questions

In [9]:
from sklearn.decomposition import PCA 

In [10]:
%%time
pca = PCA(n_components='mle') 
X_train = pca.fit_transform(X_train_tfidf)
print(X_train.shape)

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

## test

In [17]:
%%time
rf = RandomForestClassifier(n_estimators=n_estimators, verbose=2, n_jobs=-1)
rf.fit(X_train_tfidf[:1000], y_train[:1000])

building tree 1 of 10building tree 2 of 10building tree 4 of 10building tree 3 of 10



building tree 5 of 10
building tree 6 of 10building tree 7 of 10

building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
CPU times: user 11.1 s, sys: 1.62 s, total: 12.7 s
Wall time: 5.33 s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.0s finished


In [18]:
%%time
rf = RandomForestClassifier(n_estimators=n_estimators, verbose=2, n_jobs=-1)
rf.fit(X_train_tfidf[:10000], y_train[:10000])

building tree 2 of 10building tree 1 of 10building tree 3 of 10

building tree 4 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
CPU times: user 3min 28s, sys: 9.59 s, total: 3min 38s
Wall time: 1min 13s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


In [33]:
%%time
rf = RandomForestClassifier(n_estimators=n_estimators, verbose=2, n_jobs=-1)
rf.fit(X_train_tfidf[:1000], y_train[:1000])

building tree 2 of 10building tree 4 of 10building tree 1 of 10
building tree 3 of 10


building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
CPU times: user 11.5 s, sys: 2.18 s, total: 13.7 s
Wall time: 6.03 s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.7s finished


In [34]:
y = rf.predict(X_train_tfidf[:1000])

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    1.5s finished


In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_train[:1000])

0.64200000000000002

In [41]:
%%time
rf = RandomForestClassifier(n_estimators=100, verbose=2, n_jobs=-1)
rf.fit(X_train_tfidf[:1000], y_train[:1000])

building tree 1 of 100building tree 4 of 100building tree 3 of 100building tree 2 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.1s


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   43.5s finished


In [42]:
y = rf.predict(X_train_tfidf[:1000])
accuracy_score(y, y_train[:1000])

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    6.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   19.0s finished


0.86799999999999999

In [43]:
%%time
sgd = OneVsRestClassifier(SGDClassifier(loss="hinge", penalty="l2", verbose=1, n_jobs=-1))
sgd.fit(X_train_tfidf[:100], y_train[:100])

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/pyth

-- Epoch 1
Norm: 28.99, NNZs: 32, Bias: -1.069043, T: 100, Avg. loss: 0.162060
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 28.23, NNZs: 39, Bias: -1.243618, T: 200, Avg. loss: 0.127909
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 26.54, NNZs: 40, Bias: -1.243483, T: 300, Avg. loss: 0.113744
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 26.47, NNZs: 42, Bias: -1.314910, T: 400, Avg. loss: 0.100467
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 28.45, NNZs: 52, Bias: -1.593382, T: 500, Avg. loss: 0.091878
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 32.79, NNZs: 34, Bias: -1.082129, T: 100, Avg. loss: 0.076166
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 30.06, NNZs: 34, Bias: -1.082129, T: 200, Avg. loss: 0.038083
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 27.74, NNZs: 34, Bias: -1.082129, T: 300, Avg. loss: 0.025389
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 25.76, NNZs: 34, Bias: -1.082129, T: 400, Avg. loss: 0.019041
Total tra

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 227 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 228 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 229 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 230 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 232 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 233 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenv

-- Epoch 5
Norm: 28.95, NNZs: 40, Bias: -1.385550, T: 500, Avg. loss: 0.081747
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 31.08, NNZs: 23, Bias: -1.069929, T: 100, Avg. loss: 0.092558
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 29.97, NNZs: 23, Bias: -1.067775, T: 200, Avg. loss: 0.051173
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 27.66, NNZs: 23, Bias: -1.067775, T: 300, Avg. loss: 0.034116
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 25.68, NNZs: 23, Bias: -1.067775, T: 400, Avg. loss: 0.025587
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 23.97, NNZs: 23, Bias: -1.067775, T: 500, Avg. loss: 0.020469
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 32.38, NNZs: 36, Bias: -1.158864, T: 100, Avg. loss: 0.091867
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 31.27, NNZs: 36, Bias: -1.071299, T: 200, Avg. loss: 0.048041
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 28.87, NNZs: 36, Bias: -1.071299, T: 300, Avg. loss: 0.032028
Total tra

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 502 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 503 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 504 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 505 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 506 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 507 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenv

...

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1256 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1257 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1258 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1259 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1260 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1261 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virt

Total training time: 0.00 seconds.
-- Epoch 2
Norm: 28.24, NNZs: 33, Bias: -0.997626, T: 200, Avg. loss: 0.085499
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 31.54, NNZs: 42, Bias: -1.317362, T: 300, Avg. loss: 0.071317
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 29.28, NNZs: 42, Bias: -1.317362, T: 400, Avg. loss: 0.053488
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 27.33, NNZs: 42, Bias: -1.317362, T: 500, Avg. loss: 0.042790
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 29.17, NNZs: 24, Bias: -1.060321, T: 100, Avg. loss: 0.159017
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 24.83, NNZs: 24, Bias: -1.056895, T: 200, Avg. loss: 0.143081
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 26.25, NNZs: 29, Bias: -1.211150, T: 300, Avg. loss: 0.117802
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 26.57, NNZs: 29, Bias: -1.361163, T: 400, Avg. loss: 0.109342
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 24.80, NNZs: 29, Bias: -1.359277, T:

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1510 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1511 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1512 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1513 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1514 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1515 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virt


Total training time: 0.00 seconds.
-- Epoch 1
Norm: 34.08, NNZs: 34, Bias: -1.062597, T: 100, Avg. loss: 0.096317
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 31.24, NNZs: 34, Bias: -1.062597, T: 200, Avg. loss: 0.048158
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 28.83, NNZs: 34, Bias: -1.062597, T: 300, Avg. loss: 0.032106
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 26.77, NNZs: 34, Bias: -1.062597, T: 400, Avg. loss: 0.024079
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 24.98, NNZs: 34, Bias: -1.062597, T: 500, Avg. loss: 0.019263
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 31.32, NNZs: 31, Bias: -1.062715, T: 100, Avg. loss: 0.084128
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 28.71, NNZs: 31, Bias: -1.062715, T: 200, Avg. loss: 0.042064
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 28.11, NNZs: 33, Bias: -0.985555, T: 300, Avg. loss: 0.028699
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 26.57, NNZs: 33, Bias: -1.136869, T

/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1992 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1993 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1994 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1995 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1996 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virtualenvs/m/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1997 is present in all training examples.
  str(classes[c]))
/Users/lcp/.virt

In [44]:
y = sgd.predict(X_train_tfidf[:1000])
accuracy_score(y, y_train[:1000])

0.079000000000000001